<a href="https://colab.research.google.com/github/Y-Kanan/Time-Series-classification/blob/master/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}



E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 130811 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.3-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.3-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once per notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas as pd
# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import matplotlib.pyplot as plt
import copy
import numpy as np
from scipy.spatial.distance import cdist
import pandas as pd
from collections import Counter
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score,accuracy_score,recall_score,classification_report,confusion_matrix

import datetime
import math



# # ( )*96 electric Device
# train_downloaded = drive.CreateFile({'id': '1YFpv5dp67wcUuDXhBpf2srFeZs0Jjvsb'})
# train_downloaded.GetContentFile('ElectricDevices_TRAIN')
# test_downloaded = drive.CreateFile({'id': '1zNfGgi8zlmmAzJdbwv4Vj7u_hI8VgNt_'})
# test_downloaded.GetContentFile('ElectricDevices_TEST')

# (100,100)*96 ECG
train_downloaded = drive.CreateFile({'id': '1KICY5FZGMZTzDTsU0BgXYaO656WekMoi'})
train_downloaded.GetContentFile('ECG200_TRAIN.csv')
test_downloaded = drive.CreateFile({'id': '1EUIzflcLKmTeeFWkEUk_3AjVZpuDFGZs'})
test_downloaded.GetContentFile('ECG200_TEST.csv')

# # 105*144 plane
# train_downloaded = drive.CreateFile({'id': '1MNstYEkQwv9fB9clNjdtq-uoocK3iNmx'})
# train_downloaded.GetContentFile('Plane_TRAIN.csv')
# test_downloaded = drive.CreateFile({'id': '1EQPiIAwK44KuuGjqoh7tZHIO87uLAhKl'})
# test_downloaded.GetContentFile('Plane_TEST.csv')

# # (36,175)*251 Arrowhead
# train_downloaded = drive.CreateFile({'id': '1M6mSv-PO2YuP45ZoLC7PAPtwthpf8QYF'})
# train_downloaded.GetContentFile('ArrowHead_TRAIN.csv')
# test_downloaded = drive.CreateFile({'id': '17oO4YcMGHJlIrbPxWA5K9EN89vc_ns-d'})
# test_downloaded.GetContentFile('ArrowHead_TEST.csv')

# # (60)*577 car
# train_downloaded = drive.CreateFile({'id': '11jomovYRbp9C5xnzbecqyupb_OjoxxbU'})
# train_downloaded.GetContentFile('Car_TRAIN.csv')
# test_downloaded = drive.CreateFile({'id': '1iBDKLmwI-klQ7LMCXd8MPonx5jIaSnhK'})
# test_downloaded.GetContentFile('Car_TEST.csv')

# # (24,88)*350 FaceFour_image
# train_downloaded = drive.CreateFile({'id': '1OlTFCFqhrRpEbXgcKL389RubpZCKS6te'})
# train_downloaded.GetContentFile('FaceFour_TRAIN.csv')
# test_downloaded = drive.CreateFile({'id': '1XSPvnbXv6eRq-KzGv8dIMSOG-Ca9r9zC'})
# test_downloaded.GetContentFile('FaceFour_TEST.csv')

# # (30,30)*470 Beef
# train_downloaded = drive.CreateFile({'id': '1ma2Q6ddPg0mbFJqqrcxC4R2Urm92k1pS'})
# train_downloaded.GetContentFile('Beef_TRAIN.csv')
# test_downloaded = drive.CreateFile({'id': '1HOxyXYYLZ7tV7ETTXpW9kJvSwS7u6PEj'})
# test_downloaded.GetContentFile('Beef_TEST.csv')

# # ()*720 computer
# train_downloaded = drive.CreateFile({'id': '1_pOexnEpZaz-Y2NZaBxvFRw6wdoSZREA'})
# train_downloaded.GetContentFile('Computers_TRAIN.csv')
# test_downloaded = drive.CreateFile({'id': '1ydITx43z1hZQLzoHheKcgH5Srq4squzR'})
# test_downloaded.GetContentFile('Computers_TEST.csv')

#-----------------交叉验证----------------------------------------
#数据加载
data_downloaded = drive.CreateFile({'id': '1ETSMzGo6IzVLw3M0s1lqOrHxGfVd1qLh'})
data_downloaded.GetContentFile('ECG200.csv')

# 与平常read_csv无异
#数据读取
ECG_data = pd.read_csv('ECG200.csv',header=None)  
X_train,X_test,y_train,y_test=train_test_split(ECG_data.iloc[:,1:],ECG_data.iloc[:,0],test_size=0.2, random_state=None)
X_train=preprocessing.scale(X_train,axis=1) #axis=1,以行为单位运算
X_test=preprocessing.scale(X_test,axis=1)

y_test=y_test.as_matrix().reshape(-1,1)
y_train=y_train.as_matrix().reshape(-1,1)




#------------------------------------------------------------------





# #------------------不加交叉验证-------------------------------------
# # 与平常read_csv无异
# train_data = pd.read_csv('ECG200_TRAIN.csv',header=None)
# test_data = pd.read_csv('ECG200_TEST.csv',header=None)
# # train_train=train_data.iloc[71:85]
# # test_data=test_data.iloc[50:64]

# z_train=preprocessing.scale(train_data.iloc[:,1:],axis=1) #axis=1,以行为单位运算
# train_data.iloc[:,1:]=z_train
# z_test=preprocessing.scale(test_data.iloc[:,1:],axis=1)
# test_data.iloc[:,1:]=z_test
# #---------------------------------------------------------------------



# 距离矩阵用meshgrid生成
def Mydtw(x,y,window=1): #x做列，y做行
    if np.ndim(x) == 1:
        x = x.reshape(-1, 1)
    if np.ndim(y) == 1:
        y = y.reshape(-1, 1)
    r, c = np.size(x), np.size(y)
    len=r
    D0 = np.full((r + 1, c + 1),np.inf)
    D0[0,0]=0
    D1 = D0[1:, 1:]  # view
    C = D0.copy()
    #   求得D1为原始距离矩阵
    i, j = np.array(C.shape) - 2  #从倒数第二个数开始算路径
    path = [[i+1,j+1]]
    Y,X=np.meshgrid(y,x)
    D1=D0[1:,1:]=C[1:,1:]=(np.abs(X-Y))**2
    for i in range(r):
        for j in range(int(max(i - np.ceil(window * len), 0)), int(min(len , i + np.ceil(window * len)))):
            # D1[i, j] = D0[i + 1, j + 1] = C[i + 1, j + 1] = np.square(x[i] - y[j])  # 曼哈顿城市距离
            C[i + 1, j + 1] += min(C[i, j], C[i, j + 1], C[i + 1, j])  # 同时求得累计距离矩阵
    while (i > 0) or (j > 0):
        tb = np.argmin((C[i, j], C[i, j + 1], C[i + 1, j]))
        if tb == 0:
            path.append([i, j])
            i -= 1
            j -= 1
        elif tb == 1:
            path.append([i, j + 1])
            i -= 1
        else:  # (tb == 2):
            path.append([i + 1, j])
            j -= 1
    path.reverse()
    return C[-1,-1],C,path


def DDTW(s1,s2,a):
    s1_size=np.size(s1)
    s2_size=np.size(s2)
    x1=np.copy(s1)
    x2=np.copy(s2)
    for i in range(1, s1_size - 1):
        x1[i] = ((s1[i] - s1[i - 1]) + (s1[i + 1] - s1[i - 1]) / 2) / 2
    for i in range(1, s2_size - 1):
        x2[i] = ((s2[i] - s2[i - 1]) + (s2[i + 1] - s2[i - 1]) / 2) / 2
    d1,_,_=Mydtw(s1,s2,window=0.05)
    d2,_,_=Mydtw(x1,x2,window=0.05)
    dist=a*d1+(1-a)*d2
    return dist

def WDTW(x,y,window=1): #x做列，y做行
    if np.ndim(x) == 1:
        x = x.reshape(-1, 1)
    if np.ndim(y) == 1:
        y = y.reshape(-1, 1)
    r, c = np.size(x), np.size(y)
    len=r
    D0 = np.full((r + 1, c + 1),np.inf)
    D0[0,0]=0
    D1 = D0[1:, 1:]  # view
    C = D0.copy()
    #   求得D1为原始距离矩阵
    i, j = np.array(C.shape) - 2  #从倒数第二个数开始算路径
    path = [[i+1,j+1]]
    Y,X=np.meshgrid(y,x)
    D1=D0[1:,1:]=C[1:,1:]=(np.abs(X-Y))**2
    # -------------惩罚参数设置-----------------
    wmax = 1
    g = 0.05
    m = r + c / 2
    w_shape = np.shape(D1)
    wa = np.zeros(w_shape)
    for i in range(0, w_shape[0]):
        for j in range(0, i):
            wa[i, j] = wa[j, i] = wmax / (1 + math.exp(-g * (np.abs(i - j) - m / 2)))
    D1=D1*wa
    for i in range(r):
        for j in range(int(max(i - np.ceil(window * len), 0)), int(min(len , i + np.ceil(window * len)))):
            # D1[i, j] = D0[i + 1, j + 1] = C[i + 1, j + 1] = np.square(x[i] - y[j])  # 曼哈顿城市距离
            C[i + 1, j + 1] += min(C[i, j], C[i, j + 1], C[i + 1, j])  # 同时求得累计距离矩阵
    while (i > 0) or (j > 0):
        tb = np.argmin((C[i, j], C[i, j + 1], C[i + 1, j]))
        if tb == 0:
            path.append([i, j])
            i -= 1
            j -= 1
        elif tb == 1:
            path.append([i, j + 1])
            i -= 1
        else:  # (tb == 2):
            path.append([i + 1, j])
            j -= 1
    path.reverse()
    return C[-1,-1],C,path

def LCSS(A,B):
    if np.ndim(A) == 1:
        A = A.reshape(-1, 1)
    if np.ndim(B) == 1:
        B = B.reshape(-1, 1)
    c,r = np.size(A),np.size(B)
    L = np.zeros((r+1,c+1))
    for i in range(1,r+1):
        for j in range(1,c+1):
            if abs(A[j-1] - B[i-1]) < 1e-3:
                L[i,j]=L[i-1,j-1]+1
            else:
                L[i,j]=max(L[i-1,j],L[i,j-1])
                # print(L[i, j])
    # print(L)
    dist_LCSS=1-L[-1,-1]/((c+r)/2)
    return dist_LCSS,L

def reduce(x,n):
    x_shrinked=[(x[i] + x[1 + i]) / n for i in range(0, len(x) - len(x) % n -n, n)]
    x_shrinked=np.array(x_shrinked)
    return x_shrinked

def MyFastDtw(x,y,n):
    if np.ndim(x) == 1:
        x = x.reshape(-1, 1)
    if np.ndim(y) == 1:
        y = y.reshape(-1, 1)
    # 二次采样
    x_half = reduce(x,n)
    y_half = reduce(y,n)
    r, c = np.size(x_half), np.size(y_half)
    D0 = np.full((r + 1, c + 1),np.inf)
    D0[0, 0] = 0
    D1 = D0[1:, 1:]
    C=D0.copy()
    i,j=np.array(C.shape)-2
    path = [[i+1,j+1]]
    Y,X=np.meshgrid(y_half,x_half)
    D1=D0[1:,1:]=C[1:,1:]=(np.abs(X-Y))**2
    for i in range(r):
        for j in range(c):
            # D1[i, j] = D0[i + 1, j + 1] = C[i + 1, j + 1] = np.square(x[i] - y[j])  # 曼哈顿城市距离
            C[i + 1, j + 1] += min(C[i, j], C[i, j + 1], C[i + 1, j])  # 同时求得累计距离矩阵
    while (i > 0) or (j > 0):
        tb = np.argmin((C[i, j],C[i, j + 1], C[i + 1, j]))
        if tb == 0:
            path.append([i, j])
            i -= 1
            j -= 1
        elif tb == 1:
            path.append([i, j + 1])
            i -= 1
        else:  # (tb == 2):
            path.append([i + 1, j])
            j -= 1
    path.reverse()
    return C[-1,-1],path,np.shape(x_half),np.shape(y_half)

start=datetime.datetime.now()
#-----------------不包含交叉验证-------------------------
# size_train=np.shape(z_train)
# size_test=np.shape(z_test)
size_train=np.shape(X_train)
size_test=np.shape(X_test)
d=np.zeros((size_test[0],size_train[0]))
# 分类结果储存在min矩阵中 索引：最小距离，所对应的序号，分类结果，实际类别
result_min=np.full((6,size_test[0]),np.inf)
for i in range(size_test[0]):
    for j in range(size_train[0]):
#         d[i, j]= DDTW(X_test[i], X_train[j],a=0.5) # 具体度量算法 DDTW
#         d[i, j],_,_,_=MyFastDtw(X_test[i], X_train[j],n=2)  #二次采样
        d[i, j],_= LCSS(X_test[i], X_train[j])   #LCSS
#         d[i, j],_,_= WDTW(X_test[i], X_train[j])
#         d[i,j],_,_=Mydtw(X_test[i], X_train[j],window=0.05)  #DTW加限制窗口
        # d[i,j],_,xh_shape,yh_shape=f.MyFastDtw_v3(z_test[i],z_train[j],wrap=2,n=1,window=0.05)
        # print(xh_shape,yh_shape)
        if d[i,j]<result_min[0,i]:
            result_min[0,i]=d[i,j] #最小距离
            result_min[1,i]=j      #编号
            result_min[2, i] =y_train[j,0] #分类结果
    # print(path)
    result_min[3, i] = y_test[i,0]   #实际类别
    result_min[4,i] = (result_min[2,i]==result_min[3,i]) #分类是否正确
#计算准确性
accuracy = np.sum(result_min[4])/size_test[0]
print('accuracy: ',accuracy)
print(classification_report(result_min[3],result_min[2]))
end=datetime.datetime.now()

# #----------------画图----------------------------------------------------------
# plt.figure(1)
# # x=[i for i in range(0,size_test[0])]
# y1=result_min[2]
# y2=result_min[3]
# # ax=plt.subplot(111)
# plt.plot(y1,'b',label='predict')
# plt.plot(y2,'r',label='real')
# plt.title('ECG classification result using DDTW')
# plt.xlabel('test number')
# plt.ylabel('class label')
# # ymajorLocator = plt.plt.MultipleLocator(1.0) #将y轴主刻度标签设置为1.0的倍数

# plt.legend(bbox_to_anchor=(1.05, 0.5), loc=3, borderaxespad=0)

# plt.show()
# #---------------------------画图-----------------------------------------------

# d=DDTW(x,y,a=0.5)
# print('DDTW 距离为：',d)

print('runing time=',end-start)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:76: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:77: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


accuracy:  0.675
              precision    recall  f1-score   support

        -1.0       0.67      0.38      0.48        16
         1.0       0.68      0.88      0.76        24

   micro avg       0.68      0.68      0.68        40
   macro avg       0.67      0.62      0.62        40
weighted avg       0.67      0.68      0.65        40

runing time= 0:02:47.048179


In [0]:
!nvidia-smi

Sun May  5 14:08:06 2019       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 418.56       Driver Version: 410.79       CUDA Version: 10.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   65C    P0    71W / 149W |    121MiB / 11441MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
    raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [0]:
import tensorflow as tf
tf.test.gpu_device_name()




'/device:GPU:0'

In [0]:
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 12357226886647139136, name: "/device:XLA_CPU:0"
 device_type: "XLA_CPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 4051903851678770157
 physical_device_desc: "device: XLA_CPU device", name: "/device:XLA_GPU:0"
 device_type: "XLA_GPU"
 memory_limit: 17179869184
 locality {
 }
 incarnation: 1487562667885916268
 physical_device_desc: "device: XLA_GPU device", name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 11276946637
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 17520924586042218326
 physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:04.0, compute capability: 3.7"]